In [0]:
!pip install hazm

     |████████████████████████████████| 317kB 4.6MB/s 
     |████████████████████████████████| 1.4MB 17.7MB/s 
     |████████████████████████████████| 235kB 34.5MB/s 
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394470 sha256=4b085d454a15627b58656e3880d4c716a402ed37f254c779ae0b630c20d6d129
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp36-cp36m-linux_x86_64.whl size=153233 sha256=ac7f76d23ccd49ae00d3bab46144de63118a6147a288f3b4be83942117980a7e
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
import numpy as np
import pandas as pd
# from __future__ import unicode_literals
# from hazm import *
import re

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive/Razavi/Digi

/content/gdrive/My Drive/Razavi/Digi


In [0]:
df = pd.read_excel('Digi_2.xlsx')
# df.head(30)

In [0]:
df.isnull().sum()

product_id                 0
product_title              0
title_en                   0
user_id                    0
likes                      0
dislikes                   0
verification_status        0
recommend                  0
title                   3270
comment                  117
advantages             57759
disadvantages          65879
dtype: int64

In [0]:
df['title'].loc[3037]

'...'

In [0]:
#Fill NaN in title column
df['comment'].fillna('فیلد خالی است', inplace=True)
df['title'].fillna('فیلد خالی است', inplace=True)
df['advantages'].fillna('فیلد خالی است', inplace=True)
df['disadvantages'].fillna('فیلد خالی است', inplace=True)

#Replace \N in recommend column with empty value
df['recommend'] = df['recommend'].replace(r'\\N','no_idea', regex=True)

In [0]:
#transform categorical data to numeric in recommend column
df['recommend'] = df['recommend'].map({'not_recommended': int(0), 'no_idea': int(1), 'recommended': int(2)})

In [0]:
# df['comment'].loc[2515]

In [0]:
df.head(30)

,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
0,3692,ماوس بی‌سیم لاجیتک مدل M325,IT,989472,0,0,verified,1,فیلد خالی است,واقعا عالیه. من که ازش خیلی راضیم,فیلد خالی است,فیلد خالی است
1,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,3862150,4,1,verified,2,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
2,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,626843,1,0,verified,0,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,فیلد خالی است,فیلد خالی است
3,120499,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,786887,6,11,verified,1,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,67200,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,854531,19,4,verified,1,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"
5,133722,لپ تاپ 15 اینچی ایسوس مدل N501VW - A,IT,497032,6,1,verified,2,اقرار بیش از حد در ایراد گرفتن,سلام دوستان،،\r\nمنم مثه بعضی از دوستان قبل از...,"[""همه چیز که تو مشخصات اومده بجز 3 مورد کوچیک""]","[""تاچ پدش میتونست بهتر باشه از نظر سرعت پاسخ گ..."
6,148509,اسپیکر بلوتوثی دیووم مدل Bluetune bean,IT,574130,3,0,verified,2,بسیار خوب,من چند سالی هست که این اسپیکرو خریدم و واقعا ح...,"[""کیفیت ساخت بسیار خوب\r"",""کیفیت صدا بسیار عال...","[""ندارد""]"
7,56871,ادو پرفیوم زنانه امپر مدل Chifon حجم 100 میلی ...,PC,1139954,2,2,verified,1,عطر...,بوی تند ولی خوشبو داره.ماندگاریش خوبه و هر چقد...,فیلد خالی است,فیلد خالی است
8,49738,ماشین اصلاح صورت رمینگتون مدل MB4555,PA,2764879,2,1,verified,1,عمر مفید کوتاه,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...,فیلد خالی است,"[""طراحی ارگونومیک نیست\r"",""حین اصلاح مو بین تی..."
9,161548,فلاسک هنری مدل Virginia Special ظرفیت 1.9 لیتر,HW,3934937,1,0,verified,1,خوب بودممنون,خوب بودممنون,فیلد خالی است,فیلد خالی است


##Preprocessing

In [0]:
#normalize the text
normalizer = Normalizer()

def normal(text):
    text=str(text)
    text = normalizer.character_refinement(text)
    text = normalizer.punctuation_spacing(text)
    text = normalizer.affix_spacing(text)
    text = normalizer.normalize(text)
    return text

In [0]:
#find stopwords in stopwords file
stopwords = []
file = open('stopwords-fa.txt').read()
[stopwords.append(x) for x in file.split('\n')]
stopwords = set(stopwords)
len(stopwords)

def remove_stopwords(text):
    text=str(text)
    filtered_tokens = [token for token in text.split() if token not in stopwords]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [0]:
def remove_emoji(text): 
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u200c"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r' ', text)

In [0]:
def remove_link(text): 
    return re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', str(text))

In [0]:
def remove_tabs(text): 
    return re.sub(r'[\n\r\t]', '', str(text))

In [0]:
def remove_email(text): 
    return re.sub(r'\S+@\S+', '', str(text))

In [0]:
def remove_englishword(text): 
    return re.sub(r'[A-Za-z0-9]+', '', str(text))

In [0]:
def remove_chars(text): 
    # return re.sub(r'\.(?!\d)', '', str(text))
    return  re.sub(r'[$+&+;+]|[><!+،:,\(\).+]|[-+]|[…]|[\[\]»«//]|[\\]|[#+]|[_+]|[*+]|[؟+]|[?+]|[""]', '', str(text))

In [0]:
# def remove_repeatedchar(text):
#     return re.sub(r'([a-z])\1+', r'\1', 'ffffffbbbbbbbqqq')

In [0]:
def remove_extraspaces(text):
    return re.sub(r' +', ' ', text)

In [0]:
def remove_extranewlines(text):
    return re.sub(r'\n\n+', '\n\n', text)

In [0]:
#lemmatize the text
lemmatizer = Lemmatizer()

def lemma(text):
    text=str(text)
    return lemmatizer.lemmatize(text)

In [0]:
def preprocess(text):
    text = normal(text)
    text = remove_stopwords(text)
    text = remove_emoji(text)
    text = remove_link(text)
    text = remove_tabs(text) 
    text = remove_email(text) 
    text = remove_englishword(text) 
    text = remove_chars(text)
    text = lemma(text)
    text = remove_extraspaces(text) 
    text = remove_extranewlines(text) 
    return text

In [0]:
df['comment'] = df['comment'].apply(preprocess)
df['title'] = df['title'].apply(preprocess)
df['advantages'] = df['advantages'].apply(preprocess)
df['disadvantages'] = df['disadvantages'].apply(preprocess)

In [0]:
df['comment'].head(40)

0                            واقعا عالیه ازش خیلی راضیم
1     سلام قبل اینکه نظرم رو بگم میخواستم موضوع مهم ...
2     گیره های فلزی خیلی سخت میشوند لذا حوله خیلی سخ...
3     چیز رابطه ظاهر گوشی بسیار خوب بدنه یکپارچه فلز...
4     ظرفیتش براتون کافیه حتما بخرید یه شارژر ۵ ولت ...
5     سلام دوستان منم مثه بعضی دوستان قبل خرید کلی ت...
6     چند سالی هست اسپیکرو خریدم واقعا حرف نداره برن...
7     بوی تند ولی خوشبو داره ماندگاریش خوبه چقدر میگ...
8     متاسفانه عمر مفید ماشین کم هست بعد سال دیگه با...
9                                          خوب بودممنون
10    نزدیک ساله هارد رو خریدم استفاده سنگینی ازش ند...
11    بعد گذشت ۳سال ۶۰۰۰۰کیلومتر کارکرد روی ماشینم س...
12    افا وقتی اینو واسه پسرم خریدم بچه های فامیل عا...
13    پاور بانک رو حدود سال دیجی کالا گرفتم واقعا از...
14    تیغ محصول کشور آمریکا هست عکس محصول مشخص هست ت...
15    توجه قدرت خنک کنندگی حداقل ۲ ساعت زمان لازم دا...
16    امیوارم کیفیتش همینطور بمونه مدت هاس ازین برند...
17            الان حدود سالی هست دارمش خیلی راضی

In [0]:
df['disadvantages'].loc[31]

'سیم نازک گره نمیخوره ولی اگه بخره بد میخوره بیسش میتونست بهتر باشه ولی کیفیت صدا کل خوبه کار نکردن کلید های ی کم زیاد کردن صدا محصولات اپل بعد هفته خراب'

In [0]:
df.head(40)

In [0]:
df['title'][df['title'] == ' '].count()

0

In [0]:
df['title'] = df['title'].replace(r'','فیلد خالی', regex=True)
df['comment'] = df['comment'].replace(r'','فیلد خالی', regex=True)
df['advantages'] = df['advantages'].replace(r'','فیلد خالی', regex=True)
df['disadvantages'] = df['disadvantages'].replace(r'','فیلد خالی', regex=True)

In [0]:
df_clean = df.copy()

In [0]:
df_clean.to_excel("digi_clean.xlsx")  

## Feature Engineering 

In [0]:
df_cln = pd.read_excel("digi_clean2.xlsx")

In [0]:
df_cln2.describe()

,Unnamed: 0,likes,dislikes,recommend,comment_len,word_count
count,73469.000000,73469.000000,73469.000000,73469.000000,73469.000000,73469.000000
mean,48621.592713,4.239598,2.245110,1.203419,124.381263,24.019900
std,28923.818596,9.134335,10.333697,0.696272,154.320077,29.596646
min,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,23473.000000,1.000000,0.000000,1.000000,40.000000,8.000000
50%,47849.000000,2.000000,1.000000,1.000000,81.000000,16.000000
75%,73379.000000,5.000000,2.000000,2.000000,153.000000,30.000000
max,99998.000000,854.000000,1344.000000,2.000000,4859.000000,945.000000


In [0]:
df_cln.likes[df_cln.likes == 'like'].count()

34048

In [0]:
#Categorical dislikes columns

df_cln['dislikes'] = df_cln['dislikes'].astype(int)
df_cln.loc[df_cln['dislikes'] >= 1, 'dislikes'] = 'dislike'
df_cln.loc[df_cln['dislikes'] != 'dislike', 'dislikes'] = 'fewdislike'

In [0]:
#Categorical likes columns

df_cln['likes'] = df_cln['likes'].astype(int)
df_cln.loc[df_cln['likes'] >= 2, 'likes'] = 'like'
df_cln.loc[df_cln['likes'] != 'like', 'likes'] = 'fewlike'

In [0]:
df_cln.dislikes[df_cln.dislikes == 'fewdislike'].count()

49125

In [0]:
#combine two target cols into one
df_cln['target'] = df_cln['likes'].str.cat(df_cln['dislikes'],sep="-")

In [0]:
#map target column into numerical based on their wight and values
df_cln['target'] = df_cln['target'].map({'fewlike-fewdislike': int(0), 'fewlike-dislike': int(1), 'like-fewdislike': int(2), 'like-dislike': int(3)})

In [0]:
print(df_cln.target[df_cln.target == 0].count(), df_cln.target[df_cln.target == 1].count(), df_cln.target[df_cln.target == 2].count(), df_cln.target[df_cln.target == 3].count() )

12963 13525 17944 29037


In [0]:
df_cln.head(40)

,Unnamed: 0,product_title,title_en,likes,dislikes,recommend,title,comment,advantages,disadvantages,comment_len,word_count,catg_likes,catg_dislikes,target
0,1,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,like,fewdislike,2,واقعا عالیه,سلام قبل اینکه نظرم رو بگم میخواستم موضوع مهم ...,عمر طولانی افت بسیار کم میزان شارژ امپر خروجی ...,داشتدار,763,143,-75,-75,2
1,2,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,fewlike,fewdislike,0,خیلی سخت حوله اش میاد,گیره های فلزی خیلی سخت میشوند لذا حوله خیلی سخ...,فیلد خالی,فیلد خالی,228,46,-75,-75,0
2,3,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,like,dislike,1,گوشی مقرون صرفه,چیز رابطه ظاهر گوشی بسیار خوب بدنه یکپارچه فلز...,صفحه نمایش پرنور کیفیت کیفیت بالای ساخت باتری ...,کیفیت پایین اسپیکر حاشیه خالی زیر صفحه نمایش ع...,975,172,-75,-75,3
3,4,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,like,dislike,1,ابعاد استحکام نگهداری شارژ مناسب,ظرفیتش براتون کافیه حتما بخرید یه شارژر ۵ ولت ...,ظریف زیبا باریک بدنه محکم دوام,ظرفیت متوسط,292,57,-75,-75,3
4,5,لپ تاپ 15 اینچی ایسوس مدل N501VW - A,IT,like,fewdislike,2,اقرار بیش حد ایراد گرفتن,سلام دوستان منم مثه بعضی دوستان قبل خرید کلی ت...,چیز تو مشخصات اومده بجز ۳ کوچیک,تاچ پدش میتونست بهتر باشه سرعت پاسخ گویی حس کی...,984,194,-75,-75,2
5,6,اسپیکر بلوتوثی دیووم مدل Bluetune bean,IT,like,fewdislike,2,بسیار خوب,چند سالی هست اسپیکرو خریدم واقعا حرف نداره برن...,کیفیت ساخت بسیار خوب کیفیت صدا بسیار عالی باتر...,داشتدار,147,29,-75,-75,2
6,7,ادو پرفیوم زنانه امپر مدل Chifon حجم 100 میلی ...,PC,fewlike,dislike,1,عطر,بوی تند ولی خوشبو داره ماندگاریش خوبه چقدر میگ...,فیلد خالی,فیلد خالی,62,11,-75,-75,1
7,8,ماشین اصلاح صورت رمینگتون مدل MB4555,PA,fewlike,fewdislike,1,عمر مفید کوتاه,متاسفانه عمر مفید ماشین کم هست بعد سال دیگه با...,فیلد خالی,طراحی ارگونومیک نیست حین اصلاح مو تیغه ها میمو...,148,29,-75,-75,0
8,9,فلاسک هنری مدل Virginia Special ظرفیت 1.9 لیتر,HW,fewlike,fewdislike,1,خوب بودممنون,خوب بودممنون,فیلد خالی,فیلد خالی,12,2,-75,-75,0
9,10,هارددیسک اکسترنال سیگیت مدل Backup Plus Deskto...,IT,fewlike,fewdislike,2,هارد کیفیت ظاهر شیک,نزدیک ساله هارد رو خریدم استفاده سنگینی ازش ند...,کیفیت ساخت خوب ظاهر ساده زیبا صدای کم حجم زیاد,عدم مقاومت برابر ضربه عدم سهولت جا جایی دلیل آ...,351,68,-75,-75,0
